In [1]:
import datasketch 
import numpy as np
import time


In [25]:
from datasketch import WeightedMinHashGenerator

v1 = 1000 *[.5]
v2 = 1000 * [.5]
# WeightedMinHashGenerator requires dimension as the first argument
wmg = WeightedMinHashGenerator(1000, sample_size=512, seed=12)

wm1 = wmg.minhash(v1) # wm1 is of the type WeightedMinHash
wm2 = wmg.minhash(v2)
print("Estimated Jaccard is", wm1.jaccard(wm2))



Estimated Jaccard is 1.0
